In [17]:
import json
import os
import uuid
import time
from dotenv import load_dotenv
import chromadb
from chromadb import EmbeddingFunction, Documents, Embeddings
import google.generativeai as genai


In [18]:


# --- Загрузка API ключа ---
load_dotenv()
genai.configure(
    api_key=os.getenv("GEMENI_API_KEY")
)
# --- Настройки ChromaDB ---
CHROMA_PATH = "./chroma_db"
COLLECTION_NAME = "tag_docs"

# --- Функция эмбендинга ---
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        title = "tags"
        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title
        )["embedding"]

# --- Работа с ChromaDB ---
def get_chroma_db():
    chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
    return chroma_client.get_or_create_collection(name=COLLECTION_NAME, embedding_function=GeminiEmbeddingFunction())

def relevant_docs(query, db, n_results=2):
    passages = db.query(query_texts=[query], n_results=n_results)['documents'][0]
    return passages


In [ ]:
db= get_chroma_db()
db.count()

In [ ]:
question= "pupil color"
passages= relevant_docs(question, db, 5)

print(''.join(passages))


In [21]:
frequently_missed_eye_tags = {
"ringed eyes", 
 "pupils color"
}

In [ ]:
 

import mimetypes

 

def make_prompt_parts(query, relevant_passage, image_path):
    context = relevant_passage.replace("'", "").replace('"', '')

    prompt = f"""
You are a tag merging assistant. Your task is to combine input tags into a single, clean and meaningful merged tag, following the context instructions below. 
Context:
{context}

INPUT TAGS:
{query}

IMPORTANT RULES:
- Only merge according to the context.
- If for tag rules not specified in context, tag should not be changed or deleted, and should be in answer
- Your response must be ONLY the merged tag (no explanations, no punctuation, no formatting).

Merged tag:
"""

    # MIME-тип и чтение байт
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "application/octet-stream"

    with open(image_path, "rb") as f:
        image_bytes = f.read()

    image_part = {
        "mime_type": mime_type,
        "data": image_bytes
    }

    return [image_part, prompt.strip()]

def make_improved_prompt_parts(existing_tags, relevant_passage, image_path):
    context = relevant_passage.replace("'", "").replace('"', '')

    prompt = f"""
You are a tag processing assistant with TWO specific tasks:

TASK 1: Clean/merge the existing input tags according to context rules
TASK 2: Check the image for these frequently missed tags: {', '.join(frequently_missed_eye_tags)}

Context for merging rules:
{context}

EXISTING INPUT TAGS (must be preserved unless context says to merge):
{existing_tags}

CRITICAL RULES:
1. PRESERVE all input tags unless context explicitly tells you to merge them
2. For TASK 2: Only ADD missing tags if you can clearly see them in the image
3. Do NOT invent or hallucinate tags that aren't clearly visible
4. Do NOT remove input tags
5. Your response format: "existing_tags_processed, additional_tags_if_visible"

EXAMPLE:
Input: "eyeliner, eyeshadow, makeup, yellow eyes"
If image shows yellow eyes without pupils: "eyeliner, eyeshadow, makeup, yellow eyes"
If image shows yellow eyes with visible black pupils: "eyeliner, eyeshadow, makeup, yellow eyes, black pupils"

Process the tags:
"""

    # MIME-тип и чтение байт
    import mimetypes
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "application/octet-stream"

    with open(image_path, "rb") as f:
        image_bytes = f.read()

    image_part = {
        "mime_type": mime_type,
        "data": image_bytes
    }

    return [image_part, prompt.strip()]




def convert_passage_to_str(passages:list):
    return ''.join([string+"\n" for string in passages])


In [23]:
# === Цвет радужки ===
iris_colors = {
    "aqua eyes", "black eyes", "blue eyes", "brown eyes", "green eyes",
    "grey eyes", "orange eyes", "purple eyes", "pink eyes", "red eyes",
    "white eyes", "yellow eyes"
}

# === Несколько цветов радужки ===
multi_iris_colors = {
    "heterochromia", "multicolored eyes", "gradient eyes",
    "two-tone eyes", "rainbow eyes"
}

# === Форма радужки ===
iris_shapes = {
    "@ @", "mismatched irises", "dashed eyes", "Pac-man eyes",
    "ringed eyes", "squiggle eyes"
}

# === Цвет зрачков ===
pupil_colors = {
    "aqua pupils", "blue pupils", "brown pupils", "green pupils", "grey pupils",
    "orange pupils", "pink pupils", "purple pupils", "red pupils", "white pupils", "yellow pupils"
}

# === Форма зрачков ===
pupil_shapes = {
    "constricted pupils", "dilated pupils", "extra pupils", "horizontal pupils",
    "no pupils", "slit pupils", "symbol-shaped pupils", "diamond-shaped pupils",
    "flower-shaped pupils", "heart-shaped pupils", "star-shaped pupils",
    "solid circle pupils", "cross-shaped pupils", "x-shaped pupils",
    "snowflakes-shaped_pupils", "power symbol-shaped pupils", "crosshair pupils",
    "mismatched pupils"
}

# === Склера ===
sclera_colors = {
    "blue sclera", "black sclera", "blank eyes", "bloodshot eyes", "green sclera",
    "mismatched sclera", "no sclera", "orange sclera", "red sclera", "yellow sclera"
}

# === Вокруг глаз ===
around_eyes = {
    "bags under eyes", "aegyo sal", "bruised eye", "flaming eyes", "glowing eyes", "glowing eye"
}

# === Животные и нечеловеческие глаза ===
inhuman_eyes = {
    "button eyes", "cephalopod eyes", "compound eyes", "horizontal pupils",
    "lens eye", "pixel eyes"
}

# === Стилистические (эмоциональные/комедийные) глаза ===
stylistic_eyes = {
    "bulging eyes", "eye pop", "crazy eyes", "empty eyes", "dashed eyes",
    "heart-shaped eyes", "Nonowa", "solid circle eyes", "o o", "solid oval eyes",
    "0 0", "spiral-only eyes", "jitome", "tareme", "tsurime", "sanpaku", "sparkling eyes"
}

# === Серийные особенности глаз ===
series_eyes = {
    "Geass", "Sharingan", "Mangekyou Sharingan", "Rinnegan", "Byakugan"
}

# === Прочее — отражения, текст, количество глаз ===
other_eyes = {
    "eye reflection", "text in eyes", "missing eye", "one-eyed", "third eye",
    "extra eyes", "no eyes"
}

# === Эмоции и выражения ===
emotion_eyes = {
    "> <", "X3", "XD", "DX", "O o", "0 0", "3 3", "6 9", "@ @", "^ ^",
    "|_|", "= =", "+ +", ". .", "<o> <o>", "<|>_<|>"
}

# === Один/два закрытых глаза ===
closed_eyes = {
    "blinking", "closed eyes", "wince", "one eye closed", ";<", ";>", ";p"
}

# === Аксессуары и закрытые глаза ===
eye_accessories = {
    "covering own eyes", "hair over eyes", "hair over one eye", "bandage over one eye",
    "blindfold", "hat over eyes", "eyepatch", "eyelashes", "colored eyelashes",
    "fake eyelashes", "eyes visible through hair", "glasses", "makeup",
    "eyeliner", "eyeshadow", "mascara"
}

# === Взгляды (направления) ===
eye_gazes = {
    "eye contact", "looking afar", "looking around", "looking at another",
    "looking at breasts", "looking at hand", "looking at hands", "looking at mirror",
    "looking at phone", "looking at self", "looking at viewer", "looking at penis",
    "looking at pussy", "looking at crotch", "looking back", "looking down",
    "looking outside", "looking over eyewear", "looking through own legs",
    "looking to the side", "looking up"
}

# === Прочее / действия ===
misc_eyes = {
    "akanbe", "artificial eye", "glass eye", "mechanical eye", "asymmetrical eyes",
    "averting eyes", "big eyes", "blind", "partially blind", "cross-eyed",
    "drawn on eyes", "eyeball", "eye beam", "eye poke", "eye pop", "eye trail",
    "googly eyes", "half-closed eyes", "narrowed eyes", "lazy eye", "persona eyes",
    "pleading eyes", "rolling eyes", "shading eyes", "sideways glance", "squinting",
    "staring", "uneven eyes", "unusually open eyes", "upturned eyes", "wall-eyed",
    "wide-eyed", "wince"
}


In [24]:
# 1. Объединяем все множества в один
all_eye_tags = (
    misc_eyes | eye_accessories | other_eyes | stylistic_eyes | inhuman_eyes |
    around_eyes | sclera_colors | pupil_shapes | pupil_colors |
    iris_shapes | multi_iris_colors | iris_colors
)



In [ ]:
# === Пути к данным ===
image_path = r'C:\Users\liali\database_redact\test_images\__flins_genshin_impact_drawn_by_ririri_lilili_tea__da58af697048d93a25c2204f8c2e748f.jpg'
text_path = os.path.splitext(image_path)[0] + '.txt'  # тот же путь, но с .txt

# === Чтение данных ===
with open(image_path, 'rb') as f:
    image_bytes = f.read()

with open(text_path, 'r', encoding='utf-8') as f:
    raw_tags = f.read().strip()

# 2. Разделяем строку с тегами из файла (если она в виде "tag1, tag2, tag3")
raw_tag_list = [tag.strip() for tag in raw_tags.split(",")]

# 3. Фильтруем только теги, которые есть в наборах для глаз
eyes_tags_list = [tag for tag in raw_tag_list if tag in all_eye_tags]

max_docs_in_db = db.count()

merged_eye_tags_set = set(eyes_tags_list) | frequently_missed_eye_tags
merged_eye_tags = ", ".join(merged_eye_tags_set)

eyes_tags= ", ".join(eyes_tags_list)
k = min(len(merged_eye_tags)+len(frequently_missed_eye_tags), max_docs_in_db)




context = relevant_docs(merged_eye_tags, db, k)
print(context)

#model= genai.GenerativeModel('models/gemini-2.5-flash')


#answer = model.generate_content(promt)



In [ ]:
# 1. Сформировать parts
parts = make_prompt_parts(query=eyes_tags, relevant_passage= convert_passage_to_str(context), image_path=image_path)

# 2. Подключить модель
model = genai.GenerativeModel('gemini-2.5-flash')

# 3. Сгенерировать ответ
response = model.generate_content(contents=parts)

# 4. Показать результат
print(response.text)


In [44]:
from PIL import Image
current_image=Image.open(image_path)

#display(current_image)
print(eyes_tags)
print(response.text)


eyeliner, eyeshadow, makeup, yellow eyes
yellow ringed eyes with black pupils


In [54]:
def two_stage_tagging(image_path, raw_tags, db):
    """
    Этап 1: Очистка и объединение существующих тегов
    Этап 2: Проверка на недостающие теги
    """
    
    # Этап 1: Работа с существующими тегами
    raw_tag_list = [tag.strip() for tag in raw_tags.split(",")]
    eyes_tags_list = [tag for tag in raw_tag_list if tag in all_eye_tags]
    
    if eyes_tags_list:
        # Получаем контекст только для существующих тегов
        existing_tags_query = ", ".join(eyes_tags_list)
        context_existing = relevant_docs(existing_tags_query, db, min(len(eyes_tags_list) * 2, db.count()))
        
        # Промпт для объединения существующих тегов
        prompt_stage1 = f"""
You are a tag merging assistant. Your task is to clean and merge existing input tags according to the context rules.

Context:
{convert_passage_to_str(context_existing)}

INPUT TAGS (existing):
{existing_tags_query}

IMPORTANT RULES:
- Only merge/clean according to the context
- If no rules specified for a tag in context, keep it unchanged
- Do NOT add new tags that are not in the input
- Your response must be ONLY the cleaned/merged tags (no explanations)

Cleaned tags:
"""
        
        parts_stage1 = make_prompt_parts_text_only(prompt_stage1, image_path)
        model = genai.GenerativeModel('gemini-2.5-flash')
        response_stage1 = model.generate_content(contents=parts_stage1)
        stage1_result = response_stage1.text.strip()
    else:
        stage1_result = ""
    
    # Этап 2: Проверка на недостающие теги
    missing_tags_query = ", ".join(frequently_missed_eye_tags)
    context_missing = relevant_docs(missing_tags_query, db, len(frequently_missed_eye_tags))
    
    prompt_stage2 = f"""
You are analyzing an image for missing eye-related tags. 

Context about tags that are often missed:
{convert_passage_to_str(context_missing)}

Current tags from stage 1: {stage1_result}

TASK: Look at the image and check if any of these frequently missed tags should be added: {missing_tags_query}

RULES:
- Only add tags that you can clearly see in the image
- Do NOT modify existing tags from stage 1
- If you add new tags, combine them with stage 1 tags using comma or merge them if it's possible
- If no missing tags are visible, return only the stage 1 tags

Final tags:
"""
    
    parts_stage2 = make_prompt_parts_text_only(prompt_stage2, image_path)
    response_stage2 = model.generate_content(contents=parts_stage2)
    
    return response_stage2.text.strip()

def make_prompt_parts_text_only(prompt, image_path):
    """Версия без изображения для первого этапа"""
    import mimetypes
    
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "application/octet-stream"

    with open(image_path, "rb") as f:
        image_bytes = f.read()

    image_part = {
        "mime_type": mime_type,
        "data": image_bytes
    }

    return [image_part, prompt.strip()]

In [55]:
final_tags = two_stage_tagging(image_path, raw_tags, db)
print(f"Исходные теги: {raw_tags}")
print(f"Обработанные теги: {final_tags}")

Исходные теги: 1boy, antenna hair, bishounen, black hair, blue background, blue hair, border, closed mouth, earrings, expressionless, eyeliner, eyeshadow, frown, gradient background, hair between eyes, long hair, looking at viewer, looking down, makeup, male focus, multicolored hair, pale skin, portrait, purple background, red eyeliner, short hair, simple background, solo, white border, yellow eyes
Обработанные теги: eyeliner, eyeshadow, makeup, yellow ringed eyes
